In [1]:
!pip install pandas numpy requests google.colab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.9 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
import io
import os
import requests
import zipfile
from google.colab import files
from typing import Dict, Tuple

# Diretório temporário para salvar arquivos
os.makedirs("saida", exist_ok=True)

def carregar_dados(caminho: str = None, via_web: bool = True) -> pd.DataFrame:
    if via_web:
        try:
            url_raw = caminho.replace("github.com", "raw.githubusercontent.com").replace("/blob/", "/")
            response = requests.get(url_raw)
            response.raise_for_status()
            return pd.read_csv(io.StringIO(response.text))
        except Exception as e:
            raise Exception(f"Erro ao carregar arquivo da web: {e}")
    else:
        uploaded = files.upload()
        for filename in uploaded:
            return pd.read_csv(io.StringIO(uploaded[filename].decode('utf-8')))

def is_quantitativo(serie: pd.Series) -> bool:
    return pd.api.types.is_numeric_dtype(serie)

def classificar_subtipo(serie: pd.Series) -> str:
    serie = serie.dropna()
    valores_unicos = serie.unique()
    n_unique = len(valores_unicos)

    if n_unique == 1:
        return "constante"
    elif all(float(x).is_integer() for x in valores_unicos):
        return "ordinal" if n_unique <= 10 else "discreto"
    else:
        return "contínuo"

def classificar_colunas(df: pd.DataFrame) -> Tuple[pd.DataFrame, Dict[str, str]]:
    classificacoes = {}
    subtipos = {}

    for col in df.columns:
        serie = df[col]
        if is_quantitativo(serie):
            classificacoes[col] = "quantitativo"
            subtipos[col] = classificar_subtipo(serie)
        else:
            classificacoes[col] = "qualitativo"

    df_quanti = df[[col for col, tipo in classificacoes.items() if tipo == "quantitativo"]].copy()

    # Adiciona linha com os subtipos
    linha_subtipos = pd.DataFrame([subtipos], index=["_subtipo"])
    df_final = pd.concat([linha_subtipos, df_quanti], axis=0)

    return df_final, classificacoes

def salvar_arquivos(df_quanti: pd.DataFrame, classificacoes: Dict[str, str]):
    caminho_csv = "saida/dados_limpos_quanti.csv"
    caminho_html = "saida/dados_limpos_quanti.html"
    caminho_txt = "saida/classificacao_colunas.txt"
    zip_path = "saida/resultados_classificacao.zip"

    df_quanti.to_csv(caminho_csv, index=False, encoding="utf-8-sig")
    df_quanti.to_html(caminho_html, index=False)

    with open(caminho_txt, "w", encoding="utf-8") as f:
        for col, tipo in classificacoes.items():
            f.write(f"{col}: {tipo}\n")

    with zipfile.ZipFile(zip_path, 'w') as zipf:
        zipf.write(caminho_csv, arcname="dados_limpos_quanti.csv")
        zipf.write(caminho_html, arcname="dados_limpos_quanti.html")
        zipf.write(caminho_txt, arcname="classificacao_colunas.txt")

    print("Arquivos gerados e compactados:")
    files.download(zip_path)

def executar_pipeline(caminho: str = None, via_web: bool = True):
    print("▶️ Carregando dados...")
    df = carregar_dados(caminho, via_web=via_web)

    print("📊 Classificando colunas...")
    df_quanti, classificacoes = classificar_colunas(df)

    print("💾 Salvando arquivos e gerando ZIP...")
    salvar_arquivos(df_quanti, classificacoes)

# Executar a pipeline com seu link do GitHub
executar_pipeline(
    caminho="https://raw.githubusercontent.com/josecacciatore89/Dangue/refs/heads/main/dengue_resid_csv.csv",
    via_web=True
)


▶️ Carregando dados...
📊 Classificando colunas...
💾 Salvando arquivos e gerando ZIP...
Arquivos gerados e compactados:


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>